In [4]:
import numpy as np
import matplotlib.pyplot as plt

from toygrad import MLP, sigmoid

np.random.seed(2137)
m = MLP(
    sizes=[2, 3, 3],
    loss="none",
    activation=sigmoid,
    bias=True,
    batch_size=1,
    momentum=0.001,
    learning_rate=0.001,
)
CLF = "data/classification"
REG = "data/regression"

test_data = np.genfromtxt(f'{CLF}/data.simple.test.100.csv', delimiter=',')[1:]

def one_hot(y):
    y = y.astype(int)
    encoded = np.zeros((y.size, y.max()+1))
    encoded[np.arange(y.size),y] = 1
    return encoded
    

In [5]:
X = test_data[:, :-1]
Y = one_hot(test_data[:, -1])

In [6]:
loss = 0
for x, y in zip(X, Y):
    result = m.forward_pass(x)
    loss += np.sum((result ** 2) - (y ** 2))
loss = loss/len(X)
print(loss)

1.147155678145375
